In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib

In [3]:
gdsc_ge = pd.read_csv('data/Processed/gdsc_cell_ge.csv')
gdsc_dr = pd.read_csv('data/Processed/gdsc_poz_dr.csv')

In [4]:
gdsc_ge

,CCL,A1BG,A1CF,A2M,A2ML1,A3GALT2,A3GALT2P,A4GALT,A4GNT,AAAS,...,ZW10,ZWILCH,ZWINT,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,22RV1,3.537942,6.364651,5.332441,2.923030,2.815383,2.815383,3.241125,3.262633,4.722157,...,8.378382,7.526319,9.274166,3.785757,5.201929,2.742575,4.635015,3.940190,4.658663,8.088672
1,2313287,3.370950,6.284884,3.485675,2.831562,2.913369,2.913369,3.313028,3.096527,4.873621,...,8.586287,6.526713,8.021952,3.520584,4.957371,2.636933,4.350120,4.395806,4.598583,7.951146
2,5637,2.927335,2.892365,3.181651,2.926549,2.677943,2.677943,4.295357,3.205598,5.249042,...,8.498743,7.057572,9.261486,3.492602,4.813163,2.558705,4.580977,4.810975,4.371501,7.974367
3,639V,3.953010,2.858072,2.892599,2.828334,2.729762,2.729762,2.960059,3.121154,4.445200,...,8.888771,6.310088,8.021780,3.919468,5.923131,2.782005,4.969272,5.481422,5.205446,8.118109
4,647V,2.804009,2.944390,3.178071,2.675572,2.949402,2.949402,3.029297,3.024326,4.636786,...,9.365574,8.657911,9.426794,3.657576,6.082990,3.148358,5.182059,5.203430,4.806941,8.530606
5,697,5.766872,3.097477,3.427126,2.896693,2.991744,2.991744,3.418797,3.074547,4.991619,...,8.114755,7.706338,10.003868,3.467752,5.747518,2.638098,4.762875,4.673358,5.093574,7.819133
6,769P,3.051233,5.594164,3.265263,2.831614,2.780377,2.780377,3.122092,3.277065,5.033630,...,7.484788,8.439790,9.513794,3.471686,4.917818,3.299418,4.799876,5.803696,4.951508,8.092127
7,7860,3.275893,3.361768,3.463416,2.748443,2.765695,2.765695,3.999428,3.345921,4.780981,...,6.957971,6.693430,9.161835,3.567823,4.905083,2.599578,4.630182,5.432714,4.274651,7.867185
8,8MGBA,4.296319,2.941258,4.249981,2.878847,2.893562,2.893562,3.291597,2.982398,5.011438,...,6.872000,7.654035,9.567750,3.167332,4.954165,2.595022,4.830504,5.228099,4.578770,8.793368
9,8305C,5.160098,3.064121,3.242893,2.839096,2.762412,2.762412,3.494874,3.167030,5.416853,...,8.737148,7.735587,8.755837,3.804950,4.879438,2.969236,4.716554,4.669399,4.029843,7.896962


In [5]:
ccle_ge = pd.read_csv('data/Processed/ccle_cell_ge.csv')
ccle_dr = pd.read_csv('data/Processed/ccle_poz_dr.csv')

In [6]:
ccle_ge

,CCL,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,1321N1,5.542720,4.658156,3.921564,4.236747,3.535469,4.043087,4.388690,5.321489,3.997375,...,10.734462,11.162798,5.296945,5.029979,6.269173,4.294522,8.552252,7.870927,5.905405,8.140169
1,22RV1,4.753564,4.766881,7.445061,4.824879,5.714456,3.696364,4.500844,4.386674,3.917018,...,10.158100,11.869601,5.920653,5.419161,6.434936,4.582009,7.828927,6.269888,5.893025,8.193560
2,42MGBA,7.290132,5.591604,4.030574,4.113296,3.277104,3.985902,4.839848,4.578237,3.898615,...,10.125317,11.147441,5.051197,4.604050,5.914544,4.521433,8.257692,8.182643,6.077138,8.575384
3,5637,4.293473,4.510732,4.157908,4.110104,4.021707,4.169669,5.138686,6.044395,4.319823,...,9.324721,11.089995,5.511265,5.112256,5.981376,5.202768,7.276361,8.617989,5.823045,7.841768
4,639V,6.627165,4.951915,4.094668,3.980078,4.311624,4.105991,4.640191,4.547413,3.960633,...,9.624247,11.142598,6.254033,5.554727,6.485611,6.701004,8.506611,8.887660,6.401931,8.367185
5,697,6.991375,4.836096,4.027624,4.069556,4.350773,4.054177,4.691261,4.582222,3.989948,...,9.800421,11.612940,4.239479,5.218879,6.669599,4.835820,8.387289,6.824373,6.299790,8.263958
6,769P,4.477092,4.575173,5.060515,4.158888,4.104820,3.887131,4.570381,5.373354,4.130344,...,10.060470,11.064802,5.922836,5.130966,5.970377,6.865460,7.660301,9.459315,6.081260,8.333366
7,786O,4.357252,4.274907,4.001955,3.504795,3.789458,4.013966,4.885730,5.229271,4.332385,...,10.027486,11.655805,5.664288,5.047360,5.930078,4.445223,7.391838,9.634818,5.819803,8.251829
8,8MGBA,5.579644,5.416075,3.859491,5.051774,3.718749,3.707124,4.506144,4.627422,3.908807,...,9.410780,10.890116,5.154294,4.848428,6.429360,4.063601,8.159093,7.213243,5.746048,8.812535
9,8305C,6.979527,5.157835,3.990267,4.019878,4.440340,4.057010,4.730754,5.297263,4.048970,...,9.746060,10.744015,5.866875,6.072585,5.901795,6.125614,7.884105,9.000393,5.990112,7.867753


In [7]:
ctrp_ge = pd.read_csv('data/Processed/ctrp_cell_ge.csv')
ctrp_dr = pd.read_csv('data/Processed/ctrp_poz_dr.csv')

In [8]:
ctrp_ge

,CCL,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,22RV1,4.753564,4.766881,7.445061,4.824879,5.714456,3.696364,4.500844,4.386674,3.917018,...,10.158100,11.869601,5.920653,5.419161,6.434936,4.582009,7.828927,6.269888,5.893025,8.193560
1,42MGBA,7.290132,5.591604,4.030574,4.113296,3.277104,3.985902,4.839848,4.578237,3.898615,...,10.125317,11.147441,5.051197,4.604050,5.914544,4.521433,8.257692,8.182643,6.077138,8.575384
2,5637,4.293473,4.510732,4.157908,4.110104,4.021707,4.169669,5.138686,6.044395,4.319823,...,9.324721,11.089995,5.511265,5.112256,5.981376,5.202768,7.276361,8.617989,5.823045,7.841768
3,639V,6.627165,4.951915,4.094668,3.980078,4.311624,4.105991,4.640191,4.547413,3.960633,...,9.624247,11.142598,6.254033,5.554727,6.485611,6.701004,8.506611,8.887660,6.401931,8.367185
4,697,6.991375,4.836096,4.027624,4.069556,4.350773,4.054177,4.691261,4.582222,3.989948,...,9.800421,11.612940,4.239479,5.218879,6.669599,4.835820,8.387289,6.824373,6.299790,8.263958
5,769P,4.477092,4.575173,5.060515,4.158888,4.104820,3.887131,4.570381,5.373354,4.130344,...,10.060470,11.064802,5.922836,5.130966,5.970377,6.865460,7.660301,9.459315,6.081260,8.333366
6,786O,4.357252,4.274907,4.001955,3.504795,3.789458,4.013966,4.885730,5.229271,4.332385,...,10.027486,11.655805,5.664288,5.047360,5.930078,4.445223,7.391838,9.634818,5.819803,8.251829
7,8MGBA,5.579644,5.416075,3.859491,5.051774,3.718749,3.707124,4.506144,4.627422,3.908807,...,9.410780,10.890116,5.154294,4.848428,6.429360,4.063601,8.159093,7.213243,5.746048,8.812535
8,8305C,6.979527,5.157835,3.990267,4.019878,4.440340,4.057010,4.730754,5.297263,4.048970,...,9.746060,10.744015,5.866875,6.072585,5.901795,6.125614,7.884105,9.000393,5.990112,7.867753
9,8505C,4.933886,4.930040,3.864184,3.796965,4.395884,3.862075,4.778100,4.632009,4.286632,...,9.874710,11.399530,6.459784,6.269152,6.665976,7.153274,8.263125,8.346787,6.246527,8.417695


In [31]:
gdsc_ge_c = gdsc_ge.applymap(lambda x: np.nan if (x<=4) else x)

In [51]:
index = gdsc_ge_c.count() > 0.1 * gdsc_ge.shape[0]

In [52]:
choo = [k for k, v in index.items() if v]

In [54]:
gdsc_ge[choo].shape

(709, 12512)

In [56]:
gdsc_ge.loc['TT']

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A3GALT2P,A4GALT,A4GNT,AAAS,AACS,...,ZW10,ZWILCH,ZWINT,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
CCL,,,,,,,,,,,,,,,,,,,,,
TT,3.534542,2.987220,3.535791,2.905282,2.723389,2.723389,3.422541,3.229965,4.476761,3.703733,...,7.258891,8.233857,9.287881,3.404270,5.276193,3.668868,4.676271,3.671179,4.638737,7.309916
TT,4.051704,2.883121,5.150396,2.803539,2.760036,2.760036,3.017446,3.081575,4.389958,4.720932,...,7.390823,7.088810,9.363503,3.926703,5.163681,3.279762,4.549501,3.931098,4.781150,6.823483
